The code below predicts multiple molecular properties by Kernel ridge regression(KRR). The number of molecules is 109, up to 60 molecules are used as samples to "train" the KRR model, and the rest are "out-of-sample" molecules. 

In [36]:
import numpy as np;
import scipy.io;
import random;
import os;
import scipy.spatial.distance as scipy_distance

In [37]:
os.chdir('D:/tensorflow_work/retrieve_from_mysql')

In [38]:
local_dir =os.listdir()
print(local_dir)

['.git', 'KRR_multi_properties_prediction_from_MySQL.ipynb', 'molecular.mat', 'mysql.py']


In [39]:
mol_data = scipy.io.loadmat('molecular.mat');

type(mol_data)

dict

In [40]:
mol_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'id', 'xyzs0', 'atomnum', 'elementid', 'colomb_matrix', 'colomb_matrix_eigenvalues', 'r2', 'cv', 'u0', 'homo', 'lumo'])

In [41]:
def distance_matrix_of_cm_eigenvalue(cm_eigenvalue_set1,cm_eigenvalue_set2,lp=2):

        distance_matrix = scipy_distance.cdist(cm_eigenvalue_set1,cm_eigenvalue_set2,'minkowski',lp)
       
        return distance_matrix
# d_ij = lp-norm distance between the i-th vector in set1 and the j-th vector in set2. 
# when Euclidean distance, choose lp=2.0 .

In [42]:
def gaussian_k(distance_matrix,sigma):
    k_matrix = np.exp(-1.0*distance_matrix/sigma)
    return k_matrix
# k_ij = exp(-1*d_ij/sigma)

In [43]:
def inverse_k_matrix(k_matrix,lambda0):
    k_matrix_dimension = np.shape(k_matrix) 
    k_matrix = k_matrix + lambda0*np.eye(k_matrix_dimension[1])
    k_matrix = np.mat(k_matrix)      # Convert the numpy array K_matrix into a numpy matrix    
  
    inverse_k = k_matrix.I
    inverse_k = np.array(inverse_k)   
    return inverse_k
# inverse_k = (k_matrix+lambda0*I_matrix)^-1

In [44]:
def split_train_test(size,N_training,N_test):

    index_entire = list(range(size))
    random.shuffle(index_entire)
    
    index_training  = index_entire[0:N_training]
    index_test      = index_entire[N_training:N_test+N_training]
    
    return index_training,index_test     # Return the indexes of molecules for test and training

In [45]:
N_training = 60
N_test     = 30
index_training,index_test = split_train_test(109,N_training,N_test)

In [63]:
p = ['homo','lumo','r2','cv','u0'];
p_unit = ['Hatree','Hatree','Bohr^2','cal/(mol*k)','Hatree']
p_limit = [0.043,0.043,0.1,0.01,0.043]
num_p = len(p)
properties_training = np.zeros([N_training,num_p])
properties_test = np.zeros([N_test,num_p])

for i in list(range(num_p)):
    properties_training[:,i]  = np.reshape(mol_data[p[i]][index_training],[N_training,])
    properties_test[:,i]      = np.reshape(mol_data[p[i]][index_test],[N_test,])

cm_eigenvalue_training     = mol_data['colomb_matrix_eigenvalues'][index_training]
cm_eigenvalue_test         = mol_data['colomb_matrix_eigenvalues'][index_test]

In [64]:
distance_matrix_training = distance_matrix_of_cm_eigenvalue(cm_eigenvalue_training,cm_eigenvalue_training,2.0)
sigma                    = np.max(distance_matrix_training )/np.log(2.0)

inverse_k                = inverse_k_matrix(distance_matrix_training,0.0)
coeffiecients_alpha      = np.mat(inverse_k) * np.mat(properties_training)
k_test_training          = distance_matrix_of_cm_eigenvalue(cm_eigenvalue_test,cm_eigenvalue_training)

In [65]:
properties_test_predict  = np.mat(k_test_training)*coeffiecients_alpha

In [66]:
delta = np.array(properties_test-properties_test_predict)
mae   = np.sum(np.abs(delta),axis=0)/np.shape(delta)[0]
rmse  = np.sqrt(np.sum(np.square(delta),axis=0)/np.shape(delta)[0])

In [67]:
print(p)
print(p_unit)
print(mae)
print(rmse)
print(p_limit)

['homo', 'lumo', 'r2', 'cv', 'u0']
['Hatree', 'Hatree', 'Bohr^2', 'cal/(mol*k)', 'Hatree']
[  0.12646131   0.12815564  32.81402935   1.45883248   1.72713393]
[  0.17385532   0.16296989  43.35110884   1.84132207   2.48470221]
[0.043, 0.043, 0.1, 0.01, 0.043]
